In [52]:
import word2vec
import numpy
import pickle

from collections import OrderedDict

import glob
import os
import sys

from subprocess import Popen, PIPE

In [236]:
def tokenize_sentences(sentences):
    #print('Tokenizing sentences..')
    text = "".join(sentences)
    
    tokenizer = Popen(tokenizer_cmd, stdin=PIPE, stdout=PIPE)
    tok_text, _ = tokenizer.communicate(bytes(text,'utf-8'))
    
    
    finalSentences = tok_text.decode('utf-8').split('\n')[:-1]
    
    #sentencesArray = tok_text.decode('utf-8').split('.')[:-1]
    #finalSentences = []
    #for sentence in sentencesArray:
    #    finalSentences.append(sentence.split('\n'))
    return finalSentences

In [237]:
def tokenize_dictionary(sentences):
    #print('Tokenizing dictionary..')
    text = "".join(sentences)
    
    tokenizer = Popen(tokenizer_cmd, stdin=PIPE, stdout=PIPE)
    tok_text, _ = tokenizer.communicate(bytes(text,'utf-8'))
    toks = tok_text.decode('utf-8').split('\n')[:-1]
    return toks

In [238]:
def build_dict(path):
    texts = []
    currdir = os.getcwd()
    os.chdir('%s/pos/' % path)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            texts.append(f.readline().strip())
    os.chdir('%s/neg/' % path)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            texts.append(f.readline().strip())
    os.chdir(currdir)

    
    sentences = tokenize_dictionary(texts)
    
    print('Building dictionary..')
    wordcount = dict()
    for ss in sentences:
        words = ss.strip().lower().split()
        for w in words:
            if w not in wordcount:
                wordcount[w] = 1
            else:
                wordcount[w] += 1

    counts = list(wordcount.values())
    keys = list(wordcount.keys())

    sorted_idx = numpy.argsort(counts)[::-1]

    worddict = dict()

    for idx, ss in enumerate(sorted_idx):
        worddict[keys[ss]] = idx+2  # leave 0 and 1 (UNK)
 
    print(numpy.sum(counts), ' total words ', len(keys), ' unique words')

    return worddict

In [239]:
def grab_data(path, dictionary):
    texts = []
    currdir = os.getcwd()
    os.chdir(path)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            texts.append(f.readline().strip())
    os.chdir(currdir)
    
    seqs_text = []
    sentenceCount = 0
    textCount = 0
    for text in texts:
        #print(text)
        words = tokenize_sentences(text)
        seqs_sentence = []
        for idx, ss in enumerate(words):
            word = ss.strip().lower()
            wordfound = False
            for (k, v) in dictionary.items():
                if(k == word):
                    seqs_sentence.append(v)  
                    wordfound = True
                    break
            if(wordfound == False):
                seqs_sentence.append(1) 
            #sentenceCount += 1
        seqs_text.append(seqs_sentence)
        #textCount +=1
        #if(textCount >= 2): break
    print(seqs_text)
    return seqs_text
    
    return
    # sentences = tokenize(sentences)

    seqs = [None] * len(sentences)
    for idx, ss in enumerate(sentences):
        words = ss.strip().lower().split()
        seqs[idx] = [dictionary[w] if w in dictionary else 1 for w in words]

    return seqs

In [240]:
data_path = '/Users/lifa08/Documents/Lifa/Machine_Learning/Miniproject/aclImdb/'
tokenizer_cmd = ['/Users/lifa08/Documents/Lifa/Machine_Learning/Miniproject/mosesdecoder-master/scripts/tokenizer/tokenizer.perl', '-l', 'en', '-q', '-' ] 
text = ['For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.']
testOutput = tokenize_dictionary(text)
print(testOutput)

['For', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem .', 'Imagine', 'a', 'movie', 'where', 'Joe', 'Piscopo', 'is', 'actually', 'funny', '!', '', 'Maureen', 'Stapleton', 'is', 'a', 'scene', 'stealer .', 'The', 'Moroni', 'character', 'is', 'an', 'absolute', 'scream .', 'Watch', 'for', 'Alan', '', '&quot;', 'The', 'Skipper', '&quot;', '', 'Hale', 'jr.', 'as', 'a', 'police', 'Sgt.']


In [241]:
dictionary = build_dict(os.path.join(data_path, 'train'))

Building dictionary..
7078735  total words  114053  unique words


In [244]:
import pickle as pkl
f = open('imdb_own.dict.pkl', 'wb')
pkl.dump(dictionary, f, -1)
f.close()

In [245]:
train_x_pos = grab_data(data_path+'train/pos', dictionary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [246]:
print(numpy.array(train_x_pos).shape)

(12500,)


In [247]:
f = open('train_x_pos_own.pkl', 'wb')
pkl.dump((train_x_pos), f, -1)
f.close()

In [249]:
train_x_neg = grab_data(data_path+'train/neg', dictionary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [250]:
print(numpy.array(train_x_neg).shape)

(12500,)


In [251]:
train_y = [1] * len(train_x_pos) + [0] * len(train_x_neg)

In [253]:
train_x = train_x_pos + train_x_neg

In [254]:
f = open('train_x_own.pkl', 'wb')
pkl.dump((train_x, train_y), f, -1)
f.close()

In [255]:
test_x_pos = grab_data(data_path+'test/pos', dictionary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [256]:
test_x_neg = grab_data(data_path+'test/neg', dictionary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [257]:
test_y = [1] * len(test_x_pos) + [0] * len(test_x_neg)
test_x = test_x_pos + test_x_neg

f = open('test_x_own.pkl', 'wb')
pkl.dump((test_x, test_y), f, -1)
f.close()

In [189]:
def main():
    # Get the dataset from http://ai.stanford.edu/~amaas/data/sentiment/
    path = 'aclImdb'
    dictionary = build_dict(os.path.join(path, 'train'))

    train_x_pos = grab_data(path+'train/pos', dictionary)
    train_x_neg = grab_data(path+'train/neg', dictionary)
    train_x = train_x_pos + train_x_neg
    train_y = [1] * len(train_x_pos) + [0] * len(train_x_neg)

    test_x_pos = grab_data(path+'test/pos', dictionary)
    test_x_neg = grab_data(path+'test/neg', dictionary)
    test_x = test_x_pos + test_x_neg
    test_y = [1] * len(test_x_pos) + [0] * len(test_x_neg)

    f = open('imdb.pkl', 'wb')
    pkl.dump((train_x, train_y), f, -1)
    pkl.dump((test_x, test_y), f, -1)
    f.close()

    f = open('imdb.dict.pkl', 'wb')
    pkl.dump(dictionary, f, -1)
    f.close()

In [190]:
def read_dataset(path):
    """Read a dataset, where the first column contains a real-valued score,
    followed by a tab and a string of words.
    """
    dataset = []
    with open(path, "r") as f:
        for line in f:
            line_parts = line.strip().split("\t")
            dataset.append((float(line_parts[0]), line_parts[1].lower()))
    return dataset

In [191]:
sentences_train = read_dataset('train.txt')

In [192]:
f = open('sentence_forword2vector.txt', 'w')
for label, sentence in sentences_train:
    f.write(sentence)
f.close()

In [193]:
word2vec.word2phrase('sentence_forword2vector.txt', 'sentence_phrases.txt', verbose=True)

Starting training using file sentence_forword2vector.txt
Words processed: 100K     Vocab size: 76K  
Vocab size (unigrams + bigrams): 51508
Words in train file: 133711


In [194]:
word2vec.word2vec('sentence_phrases.txt', 'sentence_phrases_bin.bin', size=100, verbose=True)

Starting training using file sentence_phrases.txt
Vocab size: 3062
Words in train file: 110763


In [195]:
word2vec.word2clusters('sentence_phrases_bin.bin', 'sentence_phrases_clusters.txt', 100, verbose=True)

Starting training using file sentence_phrases_bin.bin
Vocab size: 9
Words in train file: 7240


In [199]:
model = word2vec.load('sentence_phrases_bin.bin')
model.vocab

/Users/lifa08/anaconda3/lib/python3.6/site-packages/word2vec/utils.py:7: RuntimeWarning: divide by zero encountered in true_divide
  return (1.0 / LA.norm(vec, ord=2)) * vec
/Users/lifa08/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:2168: RuntimeWarning: invalid value encountered in sqrt
  ret = sqrt(sqnorm)


array(['</s>', ',', 'the', ..., 'ethnic', 'nonsense', 'earth'],
      dtype='<U78')

In [200]:
model.vectors.shape

(3062, 100)

In [201]:
model.vectors

array([[  6.58986646e-13,   7.27593387e-13,  -6.30599360e-13, ...,
          2.50888205e-13,   5.03685635e-13,   3.18791556e-14],
       [             inf,             -inf,              inf, ...,
                     inf,             -inf,             -inf],
       [            -inf,             -inf,              inf, ...,
                    -inf,             -inf,             -inf],
       ..., 
       [             inf,             -inf,              inf, ...,
                     inf,             -inf,             -inf],
       [             inf,             -inf,              inf, ...,
                     inf,             -inf,             -inf],
       [             inf,             -inf,              inf, ...,
                     inf,             -inf,             -inf]])

In [202]:
model['nonsense'].shape

(100,)

In [203]:
model[','][:10]

array([ inf, -inf,  inf,  inf,  inf,  inf, -inf, -inf, -inf,  inf])

In [204]:
indexes, metrics = model.cosine('earth')
indexes, metrics

(array([1016, 1025, 1024, 1023, 1022, 1021, 1020, 1019, 1018, 1017]),
 array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]))

In [205]:
model.vocab[indexes]

array(['faith', 'writer', 'class', 'ultimate', 'cute', 'fears', 'fit',
       '.nothing', 'steven', 'several'],
      dtype='<U78')

In [206]:
model.generate_response(indexes, metrics)

rec.array([('faith',  nan), ('writer',  nan), ('class',  nan),
           ('ultimate',  nan), ('cute',  nan), ('fears',  nan),
           ('fit',  nan), ('.nothing',  nan), ('steven',  nan),
           ('several',  nan)], 
          dtype=[('word', '<U78'), ('metric', '<f8')])

In [207]:
model.generate_response(indexes, metrics).tolist()

[('faith', nan),
 ('writer', nan),
 ('class', nan),
 ('ultimate', nan),
 ('cute', nan),
 ('fears', nan),
 ('fit', nan),
 ('.nothing', nan),
 ('steven', nan),
 ('several', nan)]

In [208]:
indexes, metrics = model.cosine('good')
model.generate_response(indexes, metrics).tolist()

[('faith', nan),
 ('writer', nan),
 ('class', nan),
 ('ultimate', nan),
 ('cute', nan),
 ('fears', nan),
 ('fit', nan),
 ('.nothing', nan),
 ('steven', nan),
 ('several', nan)]

In [209]:
# https://radimrehurek.com/gensim/models/word2vec.html
# https://rare-technologies.com/word2vec-tutorial/

import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [210]:
sentences_train = read_dataset('train.txt')

In [211]:
print(sentences_train[:8])

[(0.69444, "the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal ."), (0.83333, "the gorgeously elaborate continuation of `` the lord of the rings '' trilogy is so huge that a column of words can not adequately describe co-writer\\/director peter jackson 's expanded vision of j.r.r. tolkien 's middle-earth ."), (0.625, 'singer\\/composer bryan adams contributes a slew of songs -- a few potential hits , a few more simply intrusive to the story -- but the whole package certainly captures the intended , er , spirit of the piece .'), (0.5, "you 'd think by now america would have had enough of plucky british eccentrics with hearts of gold ."), (0.72222, 'yet the act is still charming here .'), (0.83333, "whether or not you 're enlightened by any of derrida 's lectures on `` the other '' and `` the self , '' derrida is an undeniably fascinating and playful fellow .")

In [212]:
sentences = []
i = 0
for label, sentence in sentences_train:
    sentences.append(sentence)

print(sentences[1])

the gorgeously elaborate continuation of `` the lord of the rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director peter jackson 's expanded vision of j.r.r. tolkien 's middle-earth .


In [213]:
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [214]:
import nltk
nltk.download('popular')
# Tokenize the sentences into words

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/lifa08/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/lifa08/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/lifa08/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/lifa08/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/lifa08/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/lifa08/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

In [215]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [216]:
print(tokenized_sentences[:3])

[['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'s", 'new', '``', 'conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.'], ['the', 'gorgeously', 'elaborate', 'continuation', 'of', '``', 'the', 'lord', 'of', 'the', 'rings', "''", 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'can', 'not', 'adequately', 'describe', 'co-writer\\/director', 'peter', 'jackson', "'s", 'expanded', 'vision', 'of', 'j.r.r', '.', 'tolkien', "'s", 'middle-earth', '.'], ['singer\\/composer', 'bryan', 'adams', 'contributes', 'a', 'slew', 'of', 'songs', '--', 'a', 'few', 'potential', 'hits', ',', 'a', 'few', 'more', 'simply', 'intrusive', 'to', 'the', 'story', '--', 'but', 'the', 'whole', 'package', 'certainly', 'captures', 'the', 'intended', ',', 'er', ',', 'spirit', 'of', 'the', 'piece', '.']]


In [217]:
model = gensim.models.Word2Vec(tokenized_sentences[0:5], min_count=1, size=10, window=5, workers=4)

2018-01-17 21:56:00,583 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-01-17 21:56:00,586 : INFO : collecting all words and their counts
2018-01-17 21:56:00,587 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-17 21:56:00,589 : INFO : collected 100 word types from a corpus of 140 raw words and 5 sentences
2018-01-17 21:56:00,591 : INFO : Loading a fresh vocabulary
2018-01-17 21:56:00,593 : INFO : min_count=1 retains 100 unique words (100% of original 100, drops 0)
2018-01-17 21:56:00,596 : INFO : min_count=1 leaves 140 word corpus (100% of original 140, drops 0)
2018-01-17 21:56:00,598 : INFO : deleting the raw counts dictionary of 100 items
2018-01-17 21:56:00,602 : INFO : sample=0.001 downsamples 100 most-common words
2018-01-17 21:56:00,603 : INFO : downsampling leaves estimated 55 word corpus (40.0% of prior 140)
2018-01-17 21:56:00,605 : INFO : estimated required memory for 100 words and 10 dimensions: 5800

In [218]:
model.wv['but']

array([ 0.0302295 ,  0.03751555, -0.0090684 ,  0.03493763, -0.02394874,
        0.00456192, -0.03647238,  0.0383511 ,  0.03158452, -0.03736592], dtype=float32)

In [219]:
model.save('mygword2vmodel')

2018-01-17 21:56:08,364 : INFO : saving Word2Vec object under mygword2vmodel, separately None
2018-01-17 21:56:08,367 : INFO : not storing attribute syn0norm
2018-01-17 21:56:08,370 : INFO : not storing attribute cum_table
2018-01-17 21:56:08,374 : INFO : saved mygword2vmodel


In [220]:
model = gensim.models.Word2Vec.load('mygword2vmodel')

2018-01-17 21:56:11,198 : INFO : loading Word2Vec object from mygword2vmodel
2018-01-17 21:56:11,202 : INFO : loading wv recursively from mygword2vmodel.wv.* with mmap=None
2018-01-17 21:56:11,205 : INFO : setting ignored attribute syn0norm to None
2018-01-17 21:56:11,207 : INFO : setting ignored attribute cum_table to None
2018-01-17 21:56:11,209 : INFO : loaded mygword2vmodel


In [221]:
model.wv['but']

array([ 0.0302295 ,  0.03751555, -0.0090684 ,  0.03493763, -0.02394874,
        0.00456192, -0.03647238,  0.0383511 ,  0.03158452, -0.03736592], dtype=float32)

In [222]:
del model

In [223]:
bigram_transformer = gensim.models.Phrases(tokenized_sentences[0:5])

2018-01-17 21:56:15,443 : INFO : collecting all words and their counts
2018-01-17 21:56:15,447 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-01-17 21:56:15,451 : INFO : collected 233 word types from a corpus of 140 words (unigram + bigrams) and 5 sentences
2018-01-17 21:56:15,453 : INFO : using 233 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [224]:
model = gensim.models.Word2Vec(bigram_transformer[tokenized_sentences[0:5]], size=10, min_count=1)

/Users/lifa08/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
2018-01-17 21:56:19,342 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-01-17 21:56:19,344 : INFO : collecting all words and their counts
2018-01-17 21:56:19,347 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-17 21:56:19,350 : INFO : collected 100 word types from a corpus of 140 raw words and 5 sentences
2018-01-17 21:56:19,352 : INFO : Loading a fresh vocabulary
2018-01-17 21:56:19,355 : INFO : min_count=1 retains 100 unique words (100% of original 100, drops 0)
2018-01-17 21:56:19,357 : INFO : min_count=1 leaves 140 word corpus (100% of original 140, drops 0)
2018-01-17 21:56:19,360 : INFO : deleting the raw counts dictionary of 100 items
2018-01-17 21:5

In [225]:
model.wv['but']

array([ 0.03017185,  0.03753857, -0.00916746,  0.03481733, -0.02395921,
        0.00431288, -0.03660684,  0.03821143,  0.03154584, -0.0372339 ], dtype=float32)

In [226]:
print(model)

Word2Vec(vocab=100, size=10, alpha=0.025)


In [227]:
model.wv[tokenized_sentences[0]].shape
print(len(tokenized_sentences[0]))
del model

36


In [228]:
bigram_transformer = gensim.models.Phrases(tokenized_sentences[0:5])
model = gensim.models.Word2Vec(bigram_transformer[tokenized_sentences[0:5]], size=5, min_count=1, workers=4)

2018-01-17 21:56:29,993 : INFO : collecting all words and their counts
2018-01-17 21:56:29,996 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2018-01-17 21:56:29,998 : INFO : collected 233 word types from a corpus of 140 words (unigram + bigrams) and 5 sentences
2018-01-17 21:56:30,000 : INFO : using 233 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
/Users/lifa08/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
2018-01-17 21:56:30,002 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-01-17 21:56:30,005 : INFO : collecting all words and their counts
2018-01-17 21:56:30,007 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-17 21:56:30,010 : INFO : collected 1

In [229]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
print(len(w2v['the']))

5


In [230]:
# http://adventuresinmachinelearning.com/gensim-word2vec-tutorial/
import numpy
embedding_matrix = numpy.zeros((len(model.wv.vocab), 5))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix)

[[-0.02208557  0.09837363  0.05117133  0.02832818  0.05325192]
 [-0.08818197  0.06067047 -0.05662125 -0.08450653  0.04170386]
 [-0.03119097  0.02130773 -0.0160614  -0.02544381  0.09212921]
 [-0.05740406 -0.09070331 -0.08396011  0.02058745 -0.06510007]
 [-0.04710092 -0.00876878  0.06991214  0.02935793  0.02002611]
 [ 0.06656373  0.0385834  -0.01902515  0.09457456 -0.02702475]
 [-0.08470868  0.00165001 -0.06081249  0.02858844  0.04205706]
 [ 0.06066915  0.03912574  0.06659566 -0.01201316  0.03416824]
 [-0.07840199  0.08288328 -0.06001099  0.02987055  0.08084781]
 [-0.01090996 -0.09316173 -0.01690514 -0.04118293 -0.05039416]
 [-0.07112965 -0.08161124 -0.00730399  0.09642613  0.05540927]
 [ 0.08947527 -0.03912371  0.06929317  0.02684684  0.02780831]
 [-0.01014586 -0.02885853  0.07568201  0.04318088 -0.00745011]
 [ 0.02499827  0.06154311 -0.08528498  0.02327873  0.09503242]
 [-0.09768962 -0.02994036 -0.05794133 -0.0089996   0.03667355]
 [-0.04113455 -0.04574016  0.02809235  0.0973269   0.02

In [231]:
sentence_matrix = numpy.zeros((len(tokenized_sentences[0]), 5))
for word in tokenized_sentences[0]:
    sentence_matrix[i] = model.wv.word2index[word]
    i = i + 1
print(sentence_matrix)

AttributeError: 'EuclideanKeyedVectors' object has no attribute 'word2index'

In [125]:
numpy.array(model.wv[tokenized_sentences[0]])

array([[-0.09923361,  0.06483728, -0.06977377, -0.06283475, -0.09162995],
       [ 0.06060733, -0.00900702, -0.07504471,  0.05474842, -0.03298104],
       [-0.01017982, -0.00342942, -0.02423411,  0.01104012,  0.08833295],
       [ 0.09854638,  0.09075827,  0.03262121, -0.05288594, -0.08344082],
       [ 0.05605929, -0.04426245, -0.01464995,  0.08522175, -0.00209202],
       [-0.06084878, -0.01537897, -0.0671598 ,  0.06267373, -0.09987913],
       [-0.09923361,  0.06483728, -0.06977377, -0.06283475, -0.09162995],
       [ 0.01645874,  0.03998483, -0.03984243,  0.01195503, -0.09593436],
       [-0.05285584,  0.01166624, -0.04143319,  0.03806416,  0.07841747],
       [ 0.01958995,  0.00531222,  0.05089951,  0.08859974,  0.00560018],
       [ 0.02172123, -0.06589881,  0.00193517,  0.02630652, -0.03498821],
       [ 0.01906343, -0.04771835,  0.04810028,  0.05033817,  0.00966696],
       [ 0.04541571,  0.07379074, -0.09286577,  0.06168452, -0.02174261],
       [ 0.0807071 ,  0.07747361,  0.0

In [142]:
print(len(model.wv.vocab))

idx_sentence = numpy.zeros(len(tokenized_sentences[0]))
i = 0
for word in tokenized_sentences[0]
    idx_sentence[i] = model.wv.vocab[word].index
    i = i + 1
print(idx_sentence)

100
[  0.  13.   6.  14.   7.  15.   0.  16.  17.   4.  18.   8.  19.   9.  20.
  10.  21.   4.  22.   7.  23.   3.  24.  25.  26.  27.  28.  29.   5.  30.
  31.  32.  33.  34.  35.   2.]


In [147]:
embedding_matrix[35]

array([ 0.07591475, -0.07993622, -0.05515864, -0.01907152, -0.07367617])

In [148]:
print(tokenized_sentences.shape)

AttributeError: 'list' object has no attribute 'shape'

In [163]:
def sentence_to_idx(tokenized_sentences):
    idx_sentences = []
    for tokenized_sentence in tokenized_sentences:
        idx_one_sentence = numpy.zeros(len(tokenized_sentence))
        idx = 0
        for word in tokenized_sentence:
            idx_one_sentence[idx] = model.wv.vocab[word].index
            idx = idx + 1
        idx_sentences.append(idx_one_sentence) 
    return idx_sentences

In [164]:
idx_sentences = sentence_to_idx(tokenized_sentences[0:5])
print(idx_sentences)

[array([  0.,  13.,   6.,  14.,   7.,  15.,   0.,  16.,  17.,   4.,  18.,
         8.,  19.,   9.,  20.,  10.,  21.,   4.,  22.,   7.,  23.,   3.,
        24.,  25.,  26.,  27.,  28.,  29.,   5.,  30.,  31.,  32.,  33.,
        34.,  35.,   2.]), array([  0.,  36.,  37.,  38.,   1.,   8.,   0.,  39.,   1.,   0.,  40.,
         9.,  41.,   6.,  42.,  43.,  10.,   3.,  44.,   1.,  45.,  46.,
        47.,  48.,  49.,  50.,  51.,  52.,   4.,  53.,  54.,   1.,  55.,
         2.,  56.,   4.,  57.,   2.]), array([ 58.,  59.,  60.,  61.,   3.,  62.,   1.,  63.,  11.,   3.,  12.,
        64.,  65.,   5.,   3.,  12.,  66.,  67.,  68.,   7.,   0.,  69.,
        11.,  70.,   0.,  71.,  72.,  73.,  74.,   0.,  75.,   5.,  76.,
         5.,  77.,   1.,   0.,  78.,   2.]), array([ 79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,   1.,
        89.,  90.,  91.,  92.,  93.,   1.,  94.,   2.]), array([ 95.,   0.,  96.,   6.,  97.,  98.,  99.,   2.])]


In [ ]:
# for debugging the tokenizer script
test = b'For a movie that gets no respect there sure are a lot of memorable quotes \
listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. \
The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.'
tokenizer_cmd = ['/Users/lifa08/Documents/Lifa/Machine_Learning/Miniproject/mosesdecoder-master/scripts/tokenizer/tokenizer.perl', '-l', 'en', '-q', '-']
tokenizer = Popen(tokenizer_cmd, stdin=PIPE, stdout=PIPE)
tok_text, _ = tokenizer.communicate(test)
toks = tok_text.decode().split('\n')[:-1]
toks